## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

### Setup

In [ ]:
!pip install openai
!pip install --upgrade langchain

In [3]:
import os
#os.environ['OPENAI_API_KEY'] = ''
api_key = os.getenv('OPENAI_API_KEY')

### Imports



In [4]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

In [60]:

class HistoryQuiz():


    def __init__(self):
        self.chat = ChatOpenAI(api_key=api_key)
        self.datetimeoutputparser = DatetimeOutputParser()


    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:

            "On what date did World War 2 end?"

        '''
        system_message_prompt = SystemMessagePromptTemplate.from_template("You are going to respond with historical question related to {topic} that has a date")
        human_message_prompt = HumanMessagePromptTemplate.from_template("{topic}")
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
        prompt = chat_prompt.format_prompt(topic=topic).to_messages()
        question = self.chat(prompt)
        return question.content

    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!

        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        system_message_prompt = SystemMessagePromptTemplate.from_template("You are going to respond with only Date for the question")
        human_message_prompt = HumanMessagePromptTemplate.from_template("{topic}\n{Instruction}")
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
        prompt = chat_prompt.format_prompt(topic=question,Instruction=self.datetimeoutputparser.get_format_instructions()).to_messages()
        correct_datetime = self.chat(prompt)
        return self.datetimeoutputparser.parse(correct_datetime.content)

    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)
        year = input("Enter the year: ")
        month = input("Enter the month: ")
        day = input("Enter the day: ")
        user_datetime = datetime(int(year),int(month),int(day))

        return user_datetime


    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        return user_answer - ai_answer


### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [61]:
quiz_bot = HistoryQuiz()

                    api_key was transferred to model_kwargs.
                    Please confirm that api_key is what you intended.


In [62]:
question = quiz_bot.create_history_question(topic='World War 2')

In [63]:
question

'On what date did Germany invade Poland, officially marking the beginning of World War II?'

In [64]:
ai_answer = quiz_bot.get_AI_answer(question)

In [65]:
ai_answer

datetime.datetime(1939, 9, 1, 0, 0)

In [66]:
user_answer = quiz_bot.get_user_answer(question)

On what date did Germany invade Poland, officially marking the beginning of World War II?
Enter the year: 1938
Enter the month: 9
Enter the day: 1


In [67]:
user_answer

datetime.datetime(1938, 9, 1, 0, 0)

In [68]:
quiz_bot.check_user_answer(user_answer,ai_answer)

datetime.timedelta(days=-365)